In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 12134146
paper_name = 'steinmetz_davis_2002' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [41]:
files = ['Regression_Tc1_hom.txt','Regression_Tc2_hom.txt','Regression_Tc1_het.txt','Regression_Tc2_het.txt']

In [42]:
original_data_list = []
for f in files:
    original_data = pd.read_csv('raw_data/' + f, sep='\t')
    print('Original data dimensions: %d x %d' % (original_data.shape))
#     print(original_data1.head())
    original_data['orf'] = original_data['orf'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data.set_index('orf', inplace=True)
    
    data_cols = original_data.columns[6:]
    original_data = original_data[data_cols].apply(pd.to_numeric, axis=1, errors='coerce')
    original_data = original_data.groupby(original_data.index).mean()
    
    suffix = '_het' if f.endswith('het.txt') else '_hom'
    cols = [c+suffix for c in original_data.columns]
    original_data.columns = cols
    
    original_data_list.append(original_data)

Original data dimensions: 5918 x 12
Empty DataFrame
Columns: [gene, orf, no. of probes, measurments, F-statistic, -log(p-value), difference, YPD, YPDGE, YPG, YPE, YPL]
Index: []
Original data dimensions: 5918 x 16
Empty DataFrame
Columns: [gene, orf, no. of probes, measurments, F-statistic, -log(p-value), difference, YPD, YPDGE, YPG, YPE, YPL, YPD0.1DNP, YPD0.25DNP, YPG0.25DNP, YPG0.1DNP]
Index: []
Original data dimensions: 5918 x 12
Empty DataFrame
Columns: [gene, orf, no. of probes, measurments, F-statistic, -log(p-value), difference, YPD, YPDGE, YPE, YPL, YPG]
Index: []
Original data dimensions: 5918 x 13
Empty DataFrame
Columns: [gene, orf, no. of probes, measurments, F-statistic, -log(p-value), difference, YPD, YPG, YPD0.1DNP, YPD0.25DNP, YPG0.25DNP, YPG0.1DNP]
Index: []


In [43]:
original_data = pd.concat(original_data_list, axis=1)

In [44]:
original_data.head()

,YPD_hom,YPDGE_hom,YPG_hom,YPE_hom,YPL_hom,YPD_hom,YPDGE_hom,YPG_hom,YPE_hom,YPL_hom,...,YPDGE_het,YPE_het,YPL_het,YPG_het,YPD_het,YPG_het,YPD0.1DNP_het,YPD0.25DNP_het,YPG0.25DNP_het,YPG0.1DNP_het
orf,,,,,,,,,,,,,,,,,,,,,
YAL001C,0.95965,0.97809,1.01471,1.04274,1.00636,0.95970,0.95836,0.90337,0.98943,0.95465,...,1.02211,1.06440,1.03466,1.02400,0.96850,0.99169,0.99502,0.99279,1.01269,1.01446
YAL002W,0.98053,0.95324,0.96129,0.94241,0.96764,0.97802,0.96782,0.92258,0.91409,0.99242,...,0.89813,0.92376,0.90578,0.98536,0.94829,1.01409,0.93511,0.91650,0.99358,0.98116
YAL003W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.04541,1.06791,1.04644,1.01894,0.96731,0.98467,0.98727,0.99789,0.99432,1.00068
YAL004W,0.99940,1.01039,1.00417,0.97876,0.99019,0.96693,0.97965,1.00193,1.01351,1.00102,...,0.98429,0.97764,0.95473,0.98282,0.97040,0.99233,0.98997,0.99578,1.01083,1.00293
YAL005C,1.03910,1.11709,1.11584,1.14119,1.14626,1.03032,1.03141,1.02439,1.05416,1.01044,...,0.98936,1.09779,1.08641,1.01544,0.92760,0.96352,0.94203,0.97311,0.98071,1.00416


In [45]:
original_data = original_data.T
original_data = original_data.groupby(original_data.index).mean()
original_data = original_data.T

In [46]:
original_data.head()

,YPD0.1DNP_het,YPD0.1DNP_hom,YPD0.25DNP_het,YPD0.25DNP_hom,YPDGE_het,YPDGE_hom,YPD_het,YPD_hom,YPE_het,YPE_hom,YPG0.1DNP_het,YPG0.1DNP_hom,YPG0.25DNP_het,YPG0.25DNP_hom,YPG_het,YPG_hom,YPL_het,YPL_hom
orf,,,,,,,,,,,,,,,,,,
YAL001C,0.99502,0.95747,0.99279,0.94983,1.02211,0.968225,0.98984,0.959675,1.06440,1.016085,1.01446,0.87487,1.01269,0.89955,1.007845,0.959040,1.03466,0.980505
YAL002W,0.93511,0.99501,0.91650,0.98151,0.89813,0.960530,0.91448,0.979275,0.92376,0.928250,0.98116,0.91733,0.99358,0.94691,0.999725,0.941935,0.90578,0.980030
YAL003W,0.98727,NaN,0.99789,NaN,1.04541,NaN,1.00394,NaN,1.06791,NaN,1.00068,NaN,0.99432,NaN,1.001805,NaN,1.04644,NaN
YAL004W,0.98997,1.01472,0.99578,1.03118,0.98429,0.995020,0.98169,0.983165,0.97764,0.996135,1.00293,1.00145,1.01083,1.00670,0.987575,1.003050,0.95473,0.995605
YAL005C,0.94203,1.02089,0.97311,1.04714,0.98936,1.074250,0.96882,1.034710,1.09779,1.097675,1.00416,1.00535,0.98071,1.03409,0.989480,1.070115,1.08641,1.078350


In [50]:
# Remove the rows that are all NaNs
num_vals = original_data.notnull().sum(axis=1)

In [48]:
num_vals.unique()

array([18,  9, 15,  0,  6, 14,  5])

In [51]:
original_data = original_data.loc[num_vals > 0,:]

In [52]:
original_data.shape

(5847, 18)

In [53]:
# Normalize all datasets to YPD
# SAFE analysis indicates that taking the difference makes more sense than the ratio
cols_hom = [c for c in original_data.columns if c.endswith('_hom')]
original_data[cols_hom] = original_data[cols_hom].subtract(original_data['YPD_hom'], axis=0)

In [54]:
cols_het = [c for c in original_data.columns if c.endswith('_het')]
original_data[cols_het] = original_data[cols_het].subtract(original_data['YPD_het'], axis=0)

In [55]:
original_data.drop(columns=['YPD_hom','YPD_het'], inplace=True)

In [56]:
original_data.shape

(5847, 16)

In [59]:
dataset_ids = [4831, 4841, 4832, 4842, 4830, 4840, 4828, 4838, 4833, 4843, 4834, 4844, 4827, 4837, 4829, 4839]

In [60]:
original_data.columns = dataset_ids

# Prepare the final dataset

In [62]:
data = original_data.copy()

In [63]:
dataset_ids = original_data.columns.values
datasets = datasets.reindex(index=dataset_ids)

In [64]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [65]:
data.head()

dataset_id,4831,4841,4832,4842,4830,4840,4828,4838,4833,4843,4834,4844,4827,4837,4829,4839
data_type,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,,,,,
YAL001C,0.00518,-0.002205,0.00295,-0.009845,0.03227,0.008550,0.07456,0.056410,0.02462,-0.084805,0.02285,-0.060125,0.018005,-0.000635,0.04482,0.020830
YAL002W,0.02063,0.015735,0.00202,0.002235,-0.01635,-0.018745,0.00928,-0.051025,0.06668,-0.061945,0.07910,-0.032365,0.085245,-0.037340,-0.00870,0.000755
YAL003W,-0.01667,NaN,-0.00605,NaN,0.04147,NaN,0.06397,NaN,-0.00326,NaN,-0.00962,NaN,-0.002135,NaN,0.04250,NaN
YAL004W,0.00828,0.031555,0.01409,0.048015,0.00260,0.011855,-0.00405,0.012970,0.02124,0.018285,0.02914,0.023535,0.005885,0.019885,-0.02696,0.012440
YAL005C,-0.02679,-0.013820,0.00429,0.012430,0.02054,0.039540,0.12897,0.062965,0.03534,-0.029360,0.01189,-0.000620,0.020660,0.035405,0.11759,0.043640


## Subset to the genes currently in SGD

In [66]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [67]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,4831,4841,4832,4842,4830,4840,4828,4838,4833,4843,4834,4844,4827,4837,4829,4839
,data_type,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,,,,,
1,YAL001C,0.00518,-0.002205,0.00295,-0.009845,0.03227,0.008550,0.07456,0.056410,0.02462,-0.084805,0.02285,-0.060125,0.018005,-0.000635,0.04482,0.020830
2,YAL002W,0.02063,0.015735,0.00202,0.002235,-0.01635,-0.018745,0.00928,-0.051025,0.06668,-0.061945,0.07910,-0.032365,0.085245,-0.037340,-0.00870,0.000755
3,YAL003W,-0.01667,NaN,-0.00605,NaN,0.04147,NaN,0.06397,NaN,-0.00326,NaN,-0.00962,NaN,-0.002135,NaN,0.04250,NaN
1863,YAL004W,0.00828,0.031555,0.01409,0.048015,0.00260,0.011855,-0.00405,0.012970,0.02124,0.018285,0.02914,0.023535,0.005885,0.019885,-0.02696,0.012440
4,YAL005C,-0.02679,-0.013820,0.00429,0.012430,0.02054,0.039540,0.12897,0.062965,0.03534,-0.029360,0.01189,-0.000620,0.020660,0.035405,0.11759,0.043640


# Normalize

In [68]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [69]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [70]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,4831,4841,4832,4842,4830,4840,4828,4838,4833,4843,...,4828,4838,4833,4843,4834,4844,4827,4837,4829,4839
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
1,YAL001C,0.00518,-0.002205,0.00295,-0.009845,0.03227,0.008550,0.07456,0.056410,0.02462,-0.084805,...,1.089652,0.391115,0.759292,-0.788503,0.838680,-0.578106,0.921916,-0.087830,0.657707,0.068694
2,YAL002W,0.02063,0.015735,0.00202,0.002235,-0.01635,-0.018745,0.00928,-0.051025,0.06668,-0.061945,...,-0.160639,-0.680124,1.960731,-0.587792,2.472573,-0.358310,3.621797,-0.502857,-0.459737,-0.166592
3,YAL003W,-0.01667,NaN,-0.00605,NaN,0.04147,NaN,0.06397,NaN,-0.00326,NaN,...,0.886824,NaN,-0.037097,NaN,-0.104475,NaN,0.113237,NaN,0.609268,NaN
1863,YAL004W,0.00828,0.031555,0.01409,0.048015,0.00260,0.011855,-0.00405,0.012970,0.02124,0.018285,...,-0.415945,-0.042027,0.662743,0.116631,1.021386,0.084291,0.435263,0.144191,-0.840987,-0.029640
4,YAL005C,-0.02679,-0.013820,0.00429,0.012430,0.02054,0.039540,0.12897,0.062965,0.03534,-0.029360,...,2.131753,0.456475,1.065508,-0.301694,0.520325,-0.106962,1.028522,0.319677,2.177072,0.336036


# Print out

In [71]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [72]:
from IO.save_data_to_db3 import *

In [73]:
save_data_to_db(data_all, paper_pmid)

Deleting all datasets for PMID 12134146...


  0%|          | 0/16 [00:00<?, ?it/s]

Inserting the new data...


100%|██████████| 16/16 [02:27<00:00,  9.19s/it]

Updating the data_modified_on field...
